# Problem Statement

One of the problems that we face at Govini is mapping entities between disparate data sets. We collect data from a variety of different sources, and knowing that a certain entity is the same from one data set to another is essential.

For example, there may be a company named "FOO" in one data set, and a company named "foo 123" in another. We need to be able to determine with a high enough confidence that those are the same entity. However, a majority of the time, the data does not share a unique key. 

Using the data available, as well as relationships and metadata, we map these together with a high precision. We need you to devise an algorithm that could automatically tie related entities together. The output should have the matches as well as a confidence score for the match.

We are giving you two sample data sets, and it'll be up to you to generate a mapping between the two. In most cases, the mapping should be one to one. An ID from data set A should map to an ID in data set B. 

The sample data sets are available at the following link:
https://s3.amazonaws.com/BUCKET_FOR_FILE_TRANSFER/interview.tar.xz

# Data

The archive contains five files, described below:

## Procurement Data:

 * Company Information - data/mdl__dim_vendor.csv   (a__company.csv)
 * Location Information - data/mdl__dim_geo.csv     (a__geo.csv)

mdl__dim_vendor.csv references mdl__dim_geo.csv via the column **geo_id**.

## Finance Data:

 * Company Information - data/factset__ent_entity_coverage.csv  (b_company,csv)
 * Company Hierarchy - data/factset__ent_entity_structure.csv   (b__hierarchy.csv)
 * Location Information - data/factset__ent_entity_address.csv  (b__address.csv)

All of these files are tied together using factset_entity_id (**b_entity_id**).

The end goal of this exercise is to explore the data, and map mdl__dim_vendor.vendor_id to corresponding factset__ent_entity_coverage.factset_entity_id. Ideally, a file containing three columns: vendor_id, factset_entity_id, confidence_of_match. Please make sure there is a README file that explains how your algorithm works.

***


### Problem Statement Note
The problem statement's decription of the filenames and the data doesn't *exactly* match the sample data that was downloaded.  Unfortunately, this is *typical* of real-world scenarios, but it's close enough to still be informative.  My updates to the problem statement appear in parentheticals.

### The process of joining the datasets will be done in the following steps:

0. Setup a data analytics environment (e.g. Jupyer Lab, dtools profiler) and acquire necessary the data
1. Join a tables in geo_id and join b tables on b_entity_id
3. Profile and explore the data to determine candidate matching columns
4. Clean and normalize matching columns (e.g. upcase, truncation, normalize company name, normalize street address)
5. Determine blocking functions on exact matches for speed (e.g. STATE_PROVINCE)
6. Perform the matching
7. Emit the results

***

## Data Profiling
The CSV data was profiled using my opensource dtools (https://github.com/scholarsmate/dtools) and helped to inform my choices on features that would be suitable for blocking and matching.  The data profile results are not part of this notebook, but can be provided on request.
***

## Description Of The Matching Algorithms

### Blocking
The number of match record pairings using a naive matching approach of two tables is the number of records in the first table times the number of records in the second table.  A table with about 80,000 records being matched against another table of about 200,000 records results in 16,000,000,000 (16 trillion) pairings.  We need to vastly reduce the number of pairings to be able to perform the matching in a reasonable amount of time on modest hardware.  To reduce the pairings, we can create smaller "buckets" of records where only records in the same bucket are paired up for matching.

#### Features Selected For Blocking
For the given datasets, I chose to block on the *ISO 2 country code*, the *state/province code*, and the *first letter of the normalized company name*.  These are features that the two datasets had in common, were reasonably populated and, of the features that the datasets have in common, are typically reliable.

### Matching
Once the record pairs are blocked, we need to determine other common features that can be used for determining if a record pair should be matched and how fuzzy of a match we can tolerate.  For these datasets, I've used exact matching, where the values match only if they are identical, and Damerau–Levenshtein for fuzzy matching.  The Damerau–Levenshtein distance is the number of insertions, deletions or substitutions of a single character, or transposition of two adjacent characters, that it takes to make one value an exact match of the other.  This distance is used to compute a similarity score where we get 1 in case of and exact match and 0 in the case of complete disagreement.

#### Features Selected For Matching
The common features that I chose for blocking are normalized versions of the company name, street address, city, postal code, phone number, and website.

##### Feature Cleaning and Normalization
To provide the best chance at feature matching, the features should be pre-processed.  In pre-processing the features are cleaned up and normalized.

To normalize the company name, I used a python module called "cleanco" to remove the company's legal entity from the company name (e.g. "Foo, LLC" becomes "Foo").  To normalize the street address, I used a python module called "scourgify" that parses and rebuilds street addresses (e.g. "123 southwest Main street" becomes "123 SW MAIN ST").  Postal codes are truncated to the first 5 characters to handle matching of ZIP-5 with ZIP-9.  Phone numbers are processed by removing all non-numbers (except +).  Websites are lower-cased and those that don't begin with "http" get "http://" appended to them so that WWW.FOO.COM will match http://www.foo.com.  Company name, city and street characters are mapped into their lower-case ASCII characters, punctuation removed, white-space is elided and trimmed.

### Exact-ish Match
The first matching algorithm, I call "exact-ish" match because I'm performing exact matching on the normalized versions of the features used for matching.  If a feature is exactly matched in a record pair, that feature match scores a 1 and they don't match, it scores a 0.  This matching algorithm is very fast.  It's good for quick, high-confidence matches, but can miss matching pairs that *should* match due to minor variations in feature values.

### Fuzzy Match
The second matching algorithm uses fuzzy matching, which is slower than exact matching, but is able to put together more matching pairs because it is able to tolerate minor variations in feature values.  Fuzzy matching algorithms can be tuned for similarity, allowing more or less fuzziness for each feature comparison (e.g. allow more fuzziness for street address, but less fuzziness for ZIP-5 comparisons).  I chose an 80% similarity threshold for company name, city, postal code, phone number, and website, and I chose %75 similarity for street address.

### Match Weights
For features that match, they score a 1, and a 0 for those that don't match, but not all matches ought to be treated equal.  For example, in this case a company name match is more important (“weighs” more) than a postal code match.  To account for this, I've applied different weights to these matches in accordance with their relative importance.  I've also selected the weights so that they fit in a 0 - 100 range.  The weights are as follows:

* 60 for company name matches
* 20 for street address matches
* 5 for city matches
* 5 for postal code matches
* 5 for phone number matches
* 5 for website matches

The sum of these weights determines the “confidence of match” where 100 is a perfect match and 0 is complete disagreement.

### Match Threshold
For a record pair to be considered a match, the confidence of match must exceed a given threshold.  I chose 75 because given the assigned weights, it must always have a company match, and it must also have a street address match, or at least 3 of the 4 remaining matches (city, postal code, phone number, and website) for the record pair to be considered a match.

***

### Module imports and initialization

In [1]:
import re
import string

import numpy as np                           # BSD-3-Clause License
import unidecode                             # GNU General Public License 
import pandas as pd                          # BSD 3-Clause
import recordlinkage                         # BSD 3-Clause
import recordlinkage.preprocessing           # BSD-3-Clause License
import recordlinkage.standardise             # BSD-3-Clause License
import scourgify                             # MIT License

from werkzeug.urls import url_fix            # BSD-3-Clause License
from cleanco import prepare_terms, basename  # MIT License

#import qgrid                                 # Apache-2.0 License
import dtale                                 # GNU Lesser General Public License v2.1

# Company type terms for company name normalization
terms = prepare_terms()

## Normalization Functions

In [2]:
# Create a normalized version of a street address (the portion of the address without the city, state and zip)
def normalize_street_address(street_addr):
    if isinstance(street_addr, str) and street_addr:
        try:
            return scourgify.normalize.get_addr_line_str(scourgify.normalize.normalize_addr_str(street_addr))
        except:
            # If the street address was not parseable, return the original
            street_addr = street_addr.strip()
            if street_addr:
                return street_addr
    return np.nan

In [3]:
# Translates characters into ASCII, upper-cases, removes punctuation, elides and trims whitespace, then uses cleanco to just get the company's "basename"
def normalize_company_name(name):
    if isinstance(name, str) and name:
        return basename(unidecode.unidecode(name), terms, prefix=False, middle=False, suffix=True)
    return np.nan

In [4]:
def normalize_city_name(city):
    if isinstance(city, str) and city:
        return unidecode.unidecode(city)
    return np.nan

In [5]:
def normalize_web_address(web_address):
    if isinstance(web_address, str) and web_address:
        web_address = web_address.lower()
        if not web_address.startswith("http"):
            web_address = "http://" + web_address
        if web_address.endswith("/"):
            web_address = web_address[:-1]
        return url_fix(web_address)
    return np.nan

## Matching Functions

### Fuzzy Matching Function
The Damerau–Levenshtein distance differs from the classical Levenshtein distance by including *transpositions* among its allowable operations in addition to the three classical single-character edit operations (insertions, deletions and substitutions).

In [6]:
# min link_threshold is 5, max is 100
def link_fuzzy(a_records, b_records, link_threshold=75):
    if not 5 <= link_threshold <= 100:
        raise ValueError()
    comp = recordlinkage.Compare()
    comp.string('NAME_A_NORM', 'NAME_B_NORM', method='damerau_levenshtein', threshold=0.80, label='name_norm_score')
    comp.string('STREET_A_NORM', 'STREET_B_NORM', method='damerau_levenshtein', threshold=0.75, label='street_norm_score')
    comp.string('CITY_A_NORM', 'CITY_B_NORM', method='damerau_levenshtein', threshold=0.80, label='city_norm_score')
    comp.string('PHONE_A_NORM', 'PHONE_B_NORM', method='damerau_levenshtein', threshold=0.80, label='phone_norm_score')
    comp.string("WEB_A_NORM", "WEB_B_NORM", method='damerau_levenshtein', threshold=0.80, label='web_match_score')
    comp.string("POSTAL_A_NORM", "POSTAL_B_NORM", method='damerau_levenshtein', threshold=0.80, label='postal_code_score')

    # Block by 2-letter country code, state and the first letter of the normalized company name
    block_ix = recordlinkage.index.Block(left_on=['country_iso2', 'state', 'NAME_A_FIRST_LETTER'],
                                         right_on=['iso_country', 'state_province', 'NAME_B_FIRST_LETTER']).index(a_records, b_records)
    # Run the compare against all record pairs within a block
    comp_results = comp.compute(block_ix, a_records, b_records)
    
    # Apply weights to the matching results
    comp_results["name_norm_score"] = comp_results.name_norm_score * 60
    comp_results["street_norm_score"] = comp_results.street_norm_score * 20
    comp_results["city_norm_score"] = comp_results.city_norm_score * 5
    comp_results["phone_norm_score"] = comp_results.phone_norm_score * 5
    comp_results["web_match_score"] = comp_results.web_match_score * 5
    comp_results["postal_code_score"] = comp_results.postal_code_score * 5
    comp_results["confidence_of_match"] = comp_results[["name_norm_score", "street_norm_score", "city_norm_score", "phone_norm_score", "web_match_score", "postal_code_score"]].sum(axis=1)
    
    # Consider links to be those whose scores sum greater than or equal to the given link threshold
    return comp_results[comp_results["confidence_of_match"] >= link_threshold]

### Exact-ish Matching Function

In [7]:
# min link_threshold is 5, max is 100
def link_exactish(a_records, b_records, link_threshold=75):
    if not 5 <= link_threshold <= 100:
        raise ValueError()
    comp = recordlinkage.Compare()
    comp.exact('NAME_A_NORM', 'NAME_B_NORM', label='name_norm_score')
    comp.exact('STREET_A_NORM', 'STREET_B_NORM', label='street_norm_score')
    comp.exact('CITY_A_NORM', 'CITY_B_NORM', label='city_norm_score')
    comp.exact('PHONE_A_NORM', 'PHONE_B_NORM', label='phone_norm_score')
    comp.exact("WEB_A_NORM", "WEB_B_NORM", label='web_match_score')
    comp.exact("POSTAL_A_NORM", "POSTAL_B_NORM", label='postal_code_score')

    # Block by state code, DOB, gender, ZIP 5 and phonetic first name to create pairs of candidate links
    block_ix = recordlinkage.index.Block(left_on=['country_iso2', 'state', 'NAME_A_FIRST_LETTER'],
                                         right_on=['iso_country', 'state_province', 'NAME_B_FIRST_LETTER']).index(a_records, b_records)
    # Run the compare against all record pairs within a block
    comp_results = comp.compute(block_ix, a_records, b_records)

    # Apply weights to the matching results
    comp_results["name_norm_score"] = comp_results.name_norm_score * 60
    comp_results["street_norm_score"] = comp_results.street_norm_score * 20
    comp_results["city_norm_score"] = comp_results.city_norm_score * 5
    comp_results["phone_norm_score"] = comp_results.phone_norm_score * 5
    comp_results["web_match_score"] = comp_results.web_match_score * 5
    comp_results["postal_code_score"] = comp_results.postal_code_score * 5
    comp_results["confidence_of_match"] = comp_results[["name_norm_score", "street_norm_score", "city_norm_score", "phone_norm_score", "web_match_score", "postal_code_score"]].sum(axis=1)
    
    # Consider links to be those whose scores sum greater than or equal to the given link threshold
    return comp_results[comp_results["confidence_of_match"] >= link_threshold]

### Read the A-side company data

In [8]:
df_a_company = pd.read_csv("data/a__company.csv", index_col="vendor_id", usecols=["vendor_id", "name", "address", "phone", "websiteurl", "geo_id"])[["name", "address", "phone", "websiteurl", "geo_id"]]
#qgrid.show_grid(df_a_company)
dtale.show(df_a_company)

2020-11-11 21:26:44,990 - INFO     - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2020-11-11 21:26:44,991 - INFO     - NumExpr defaulting to 8 threads.


### Read the A-side location data

In [9]:
df_a_geo = pd.read_csv("data/a__geo.csv", index_col="geo_id", usecols=["geo_id", "country_iso2", "city", "state", "zipcode"], dtype={'zipcode': 'str'})[["country_iso2", "city", "state", "zipcode"]]
dtale.show(df_a_geo)

### Join the A-side company and location data into a single A-side table, then drop the geo_id column

In [10]:
df_a = pd.merge(df_a_company, df_a_geo, on="geo_id", right_index=True) # left inner-join

df_a.drop(columns=["geo_id"], inplace=True)

### Clean and normalize the A-side matching features

In [11]:
df_a["NAME_A_NORM"] = recordlinkage.standardise.clean(df_a.name.apply(normalize_company_name))
df_a["NAME_A_FIRST_LETTER"] = df_a.NAME_A_NORM.apply(lambda x: x[:1] if x else '')
df_a["STREET_A_NORM"] = recordlinkage.standardise.clean(df_a.address.apply(normalize_street_address))
df_a["CITY_A_NORM"] = recordlinkage.standardise.clean(df_a.city.apply(normalize_city_name))
df_a["PHONE_A_NORM"] = recordlinkage.standardise.phonenumbers(df_a.phone)
df_a["WEB_A_NORM"] = df_a.websiteurl.apply(normalize_web_address)
df_a["POSTAL_A_NORM"] = df_a.zipcode.apply(lambda x: x[:5].upper() if x and isinstance(x, str) else np.nan)

dtale.show(df_a)

In [12]:
print(len(df_a_geo))
print(len(df_a_company))
print(len(df_a))

14480
76344
76344


### Read the B-side company data

In [13]:
df_b_company = pd.read_csv("data/b__company.csv", index_col="b_entity_id", usecols=["b_entity_id", "entity_proper_name", "web_site"])[["entity_proper_name", "web_site"]]
dtale.show(df_b_company)

### Read the B-side location data

In [14]:
df_b_address = pd.read_csv("data/b__address.csv", index_col="b_entity_id", usecols=["b_entity_id", "iso_country", "location_city", "state_province", "location_postal_code", "location_street1", "location_street2", "location_street3", "tele_full"])[["iso_country", "location_city", "state_province", "location_postal_code", "location_street1", "location_street2", "location_street3", "tele_full"]]
dtale.show(df_b_address)

### Join the B-side company and location data into a single B-side table, since there are now duplicate b_entity_ids, we need to create a new table index

In [15]:
df_b = pd.merge(df_b_company, df_b_address, on="b_entity_id", how="left")

df_b = df_b.reset_index()
df_b.index.names=["pk_b"]

### Clean and normalize the B-side matching features

In [16]:
# We can't concatenate with nan's, so change these to empty strings
df_b.location_street1 = df_b.location_street1.fillna('')
df_b.location_street2 = df_b.location_street2.fillna('')
df_b.location_street3 = df_b.location_street3.fillna('')

df_b["NAME_B_NORM"] = recordlinkage.standardise.clean(df_b.entity_proper_name.apply(normalize_company_name))
df_b["STREET_B"] = df_b.location_street1 + " " + df_b.location_street2 + " " + df_b.location_street3
df_b["NAME_B_FIRST_LETTER"] = df_b.NAME_B_NORM.apply(lambda x: x[:1] if x else '')
df_b["STREET_B_NORM"] = recordlinkage.standardise.clean(df_b.STREET_B.apply(normalize_street_address))
df_b["CITY_B_NORM"] = recordlinkage.standardise.clean(df_b.location_city.apply(normalize_city_name))
df_b["PHONE_B_NORM"] = recordlinkage.standardise.phonenumbers(df_b.tele_full)
df_b["WEB_B_NORM"] = df_b.web_site.apply(normalize_web_address)
df_b["POSTAL_B_NORM"] = df_b.location_postal_code.apply(lambda x: x[:5].upper() if x and isinstance(x, str) else np.nan)

df_b.drop(columns=["STREET_B"], inplace=True)

print(len(df_b_company))
print(len(df_b))
#df_b.to_csv("data/b.csv")
dtale.show(df_b)

196498
200239


### Run the exact-ish match

In [17]:
#%%timeit
exactish_result = link_exactish(df_a, df_b)
dtale.show(exactish_result)

In [18]:
len(exactish_result)

2268

In [19]:
exactish_result.reset_index(level=0, inplace=True)

exactish_joined = pd.merge(exactish_result, df_b, on="pk_b")
exactish_joined = pd.merge(exactish_joined, df_a, on="vendor_id")

#exactish_joined["vendor_id"] = exactish_result.index[1]
exactish_joined.to_csv("results/exactish_joined.csv")
exactish_joined_trimmed = exactish_joined[['NAME_A_NORM', 'NAME_B_NORM', 'STREET_A_NORM', 'STREET_B_NORM', 'CITY_A_NORM', 'CITY_B_NORM', "POSTAL_A_NORM", "POSTAL_B_NORM", 'PHONE_A_NORM', 'PHONE_B_NORM', "WEB_A_NORM", "WEB_B_NORM", "vendor_id", "b_entity_id", "confidence_of_match", "name_norm_score", "street_norm_score", "city_norm_score", "phone_norm_score", "web_match_score", "postal_code_score"]]
exactish_joined_trimmed.to_csv("results/exactish_joined_trimmed.csv")
dtale.show(exactish_joined_trimmed)

### Run the fuzzy match

In [ ]:
#%%timeit
fuzzy_result = link_fuzzy(df_a, df_b)
dtale.show(fuzzy_result)

In [ ]:
len(fuzzy_result)

In [ ]:
fuzzy_result.reset_index(level=0, inplace=True)

fuzzy_joined = pd.merge(fuzzy_result, df_b, on="pk_b")
fuzzy_joined = pd.merge(fuzzy_joined, df_a, on="vendor_id")

fuzzy_joined.to_csv("results/fuzzy_joined.csv")
fuzzy_joined_trimmed = fuzzy_joined[['NAME_A_NORM', 'NAME_B_NORM', 'STREET_A_NORM', 'STREET_B_NORM', 'CITY_A_NORM', 'CITY_B_NORM', "POSTAL_A_NORM", "POSTAL_B_NORM", 'PHONE_A_NORM', 'PHONE_B_NORM', "WEB_A_NORM", "WEB_B_NORM", "vendor_id", "b_entity_id", "confidence_of_match", "name_norm_score", "street_norm_score", "city_norm_score", "phone_norm_score", "web_match_score", "postal_code_score"]]
fuzzy_joined_trimmed.to_csv("results/fuzzy_joined_trimmed.csv")
dtale.show(fuzzy_joined_trimmed)

### Emit final results

In [ ]:
exactish_pairs = exactish_joined[["vendor_id", "b_entity_id", "confidence_of_match"]]
exactish_pairs.to_csv("results/exactish_pairs.csv")
fuzzy_pairs = fuzzy_joined[["vendor_id", "b_entity_id", "confidence_of_match"]]
fuzzy_pairs.to_csv("results/fuzzy_pairs.csv")

### Done!
Review the results in the results folder.